<a href="https://colab.research.google.com/github/TakuroTerui/PyOptBook/blob/main/%E6%95%B0%E7%90%86%E6%9C%80%E9%81%A9%E5%8C%96_ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ohmsha/PyOptBook

Cloning into 'PyOptBook'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 206 (delta 82), reused 71 (delta 46), pack-reused 76
Receiving objects: 100% (206/206), 2.31 MiB | 11.13 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [2]:
cd PyOptBook

/content/PyOptBook


### 2-1 連立方程式をPythonの数理最適化ライブラリで解く

In [4]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 110.9 MB/s eta 0:00:00


In [5]:
import pulp

# SLEは任意の名前
problem = pulp.LpProblem('SLE', pulp.LpMaximize)

x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

problem += 120 * x + 150 * y == 1440
problem += x + y == 10

status = problem.solve()

print('Starus:', pulp.LpStatus[status])
print('x=', x.value(), 'y=', y.value())

Starus: Optimal
x= 2.0 y= 8.0


### 2-2 線形計画問題をPythonの数理最適化ライブラリで解く

In [6]:
problem = pulp.LpProblem('LP', pulp.LpMaximize)

x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

problem += 1 * x + 3 * y <= 30
problem += 2 * x + 1 * y <= 40
problem += x >= 0
problem += y >= 0
problem += x + 2 * y

status = problem.solve()

print('status:', pulp.LpStatus[status])
print('x=', x.value(), 'y=', y.value(), 'obj=', problem.objective.value())

status: Optimal
x= 18.0 y= 4.0 obj= 26.0


### 2-3 規模の大きな数理最適化問題をPythonの数理最適化ライブラリで解く

In [8]:
import pandas as pd
import pulp

stock_df = pd.read_csv('2.tutorial/stocks.csv')
stock_df

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [9]:
require_df = pd.read_csv('2.tutorial/requires.csv')
require_df

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


In [10]:
gain_df = pd.read_csv('2.tutorial/gains.csv')
gain_df

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [20]:
P = gain_df['p'].tolist()
print(P)

M = stock_df['m'].tolist()
print(M)

['p1', 'p2', 'p3', 'p4']
['m1', 'm2', 'm3']


In [12]:
stock = {row.m:row.stock for row in stock_df.itertuples()}
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [13]:
stock = dict(zip(stock_df['m'], stock_df['stock']))
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [14]:
stock = dict((row.m, row.stock) for row in stock_df.itertuples())
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [15]:
stock = stock_df.set_index('m').to_dict()['stock']
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [16]:
require = {(row.p, row.m):row.require for row in require_df.itertuples()}
print(require)

{('p1', 'm1'): 2, ('p1', 'm2'): 0, ('p1', 'm3'): 1, ('p2', 'm1'): 3, ('p2', 'm2'): 2, ('p2', 'm3'): 0, ('p3', 'm1'): 0, ('p3', 'm2'): 2, ('p3', 'm3'): 2, ('p4', 'm1'): 2, ('p4', 'm2'): 2, ('p4', 'm3'): 2}


In [17]:
gain = {row.p:row.gain for row in gain_df.itertuples()}
print(gain)

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}


In [18]:
problem = pulp.LpProblem('LP2', pulp.LpMaximize)

In [23]:
x = pulp.LpVariable.dicts('x', P, cat='Continuous')
print(x)

{'p1': x_p1, 'p2': x_p2, 'p3': x_p3, 'p4': x_p4}


In [24]:
for p in P:
    problem += x[p] >= 0

In [25]:
for m in M:
    problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]

In [26]:
problem += pulp.lpSum([gain[p] * x[p] for p in P])

In [27]:
status = problem.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [28]:
for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

p1 12.142857
p2 3.5714286
p3 7.4285714
p4 0.0
obj= 80.42857099999999


In [29]:
import pandas as pd
import pulp

# データの取得
require_df = pd.read_csv('2.tutorial/requires.csv')
stock_df = pd.read_csv('2.tutorial/stocks.csv')
gain_df = pd.read_csv('2.tutorial/gains.csv')

# リストの定義
P = gain_df['p'].tolist()
M = stock_df['m'].tolist()

# 定数の定義
stock = {row.m:row.stock for row in stock_df.itertuples()}
gain = {row.p:row.gain for row in gain_df.itertuples()}
require = {(row.p, row.m):row.require for row in require_df.itertuples()}

# 数理最適化モデルの定義
# Before
# problem = pulp.LpProblem('LP2', pulp.LpMaximize)
# After
problem = pulp.LpProblem('IP', pulp.LpMaximize)

# 変数の定義
# Before
# x = pulp.LpVariable.dict('x', P, cat='Continuous')
# After
x = pulp.LpVariable.dict('x', P, cat='Integer')

# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum((require[p, m] * x[p] for p in P)) <= stock[m]

# 目的関数の定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

Status: Optimal
p1 13.0
p2 3.0
p3 7.0
p4 -0.0
obj= 79.0
